In [1]:
from datetime import datetime
import datetime as dt
import xarray as xr
import numpy as np
import glob
from dask.distributed import Client
import gc
import dask

from madmex.recipes import RECIPES
from datacube.api import GridWorkflow

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from django.contrib.gis.geos.geometry import GEOSGeometry
import json
from madmex.models import PredictObject



from operator import itemgetter

from affine import Affine
from rasterio.features import rasterize
from collections import OrderedDict
import fiona
import scipy
import pandas as pd


from madmex.overlay.conversions import rasterize_xarray
from madmex.util import chunk

from sklearn.cluster import KMeans

import geopandas as gpd
from madmex.util import join_dicts

from madmex.util.spatial import get_geom_bbox

from shapely.geometry import Polygon, mapping
import shapely.wkt
from shapely.geometry import box
from madmex.indexing import wkt_to_proj4
from pyproj import CRS
from affine import Affine
import rasterio

import geopandas as gpd
from sklearn.cluster import KMeans
import pickle

## Clusterization with many dctiles randomly sampled 
To fit models, set `fit = True` and run 1) for all dctiles and save their gdf_df_join's. After that, run 2) and 3)

To do predictions, set `fit = False` and you have tu run 1) one time for a dctile in particular to get the gdf_df_join and 4) for the predictions (this part can run many times with different models, just use the right features of gdf_df_join)

To do csv's for eda, run 5) and 6)

In [2]:
tiles = ['39_-30','39_-31','39_-32','40_-30','41_-30','41_-31','41_-32']

In [3]:
tile = tiles[0]

In [4]:
path = '/shared_volume/datacube/datacube_ingest/recipes/recipe_mex_s2_10m_scl_crops_10_18_09_19'

In [5]:
sr_0_no_chunks = xr.open_mfdataset(path +'/s2_10m_scl_crops_{}_2019-04-01.nc'.format(tile),
                                  data_vars='minimal', coords='minimal',
                                  combine='by_coords')

In [6]:
# fraction of "pixels" in combined that are going to be randomly sampled
frac = 0.35

In [ ]:
# if fit is true the samples are going to be used to fit a model. If fit is false, then the samples are for prediction
fit = False

In [ ]:
# number of clusters in unsupervised clusterization
n_clusters = 30

In [11]:
query_path = '/shared_volume/segmentation_antares3_results/bis_seg_mex_s2_10m_scl_10_18_09_19_sentinel_2_{}_2019.shp'.format(tile)
query_path

'/shared_volume/segmentation_antares3_results/bis_seg_mex_s2_10m_scl_10_18_09_19_sentinel_2_41_-32_2019.shp'

In [13]:
dimensions_dataset = list(sr_0_no_chunks.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [14]:
out_shape_x = sr_0_no_chunks.sizes[xdim]
out_shape_y = sr_0_no_chunks.sizes[ydim]

In [15]:
c, a, b, f, d, e = sr_0_no_chunks.crs.GeoTransform
aff = Affine(*[a,b,c,d,e,f])
crs = wkt_to_proj4(sr_0_no_chunks.crs.crs_wkt)

In [17]:
path = query_path
with fiona.open(path) as src:
    fc = list(src)
    fc_subset=[(x['properties']['id'], x['geometry']) for x in fc]

In [18]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], crs = crs)
gdf = gdf.rename(columns={'id':'features_id'})
# Deallocate geoarray and feature collection
    
fc_sorted = sorted(fc_subset, key=itemgetter(0))
iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])

In [19]:
fc_raster = rasterize(iterable_zip, transform=aff,
                      out_shape=(out_shape_x, out_shape_y),
                      dtype='float64', fill=np.nan)
    
fc_raster = fc_raster.astype(int)
fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

In [20]:
fc_subset = gdf = fc_sorted = iterable_zip = fc_raster = None

In [21]:
sr_0_no_chunks = sr_0_no_chunks.drop(['crs'])

In [22]:
combined = xr.merge([fc_dataarray, sr_0_no_chunks]) 

In [23]:
combined = combined.stack(z=(xdim, ydim))
combined = combined.reset_index('z')
combined = combined.drop([xdim,ydim])
combined = combined.where((np.isfinite(combined['features_id'])), drop=True)

In [24]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 5.67e+05 5.653e+05 5.653e+05 ... 673.0 673.0 673.0
    blue_mean    (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    green_mean   (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    red_mean     (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    nir_mean     (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    ndvi_mean    (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    ndvi_max     (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    ndvi_min     (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>
    ndvi_argmax  (z) float64 dask.array<chunksize=(25020004,), meta=np.ndarray>

In [ ]:
if fit:
    num_samples = int(int(np.max(combined.features_id).values) * frac) # 0.07 is almost 1/15
    combined = combined.where(combined.features_id.isin(np.random.choice(int(np.max(combined.features_id).values)+1, num_samples, 
                                                          replace=False)), drop=True)
else:
    seg_intersec_path = '/shared_volume/tasks/2019/clusterization_Veracruz/seg_mex_s2_10m_scl_10_18_09_19_intersect_shp_puntos_martinez.shp'
    with fiona.open(seg_intersec_path) as src:
        fc1 = list(src)
    features_id_l = [x['properties']['id'] for x in fc1]
    features_id_list = list(set(features_id_l))
    fc1 = None
    len(features_id_list)
    combined = combined.where(combined.features_id.isin(features_id_list), drop=True)

In [29]:
combined

<xarray.Dataset>
Dimensions:      (z: 71907)
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 5.24e+05 5.24e+05 5.24e+05 ... 1.451e+05 1.451e+05
    blue_mean    (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    green_mean   (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    red_mean     (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    nir_mean     (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    ndvi_mean    (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    ndvi_max     (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    ndvi_min     (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>
    ndvi_argmax  (z) float64 dask.array<chunksize=(71907,), meta=np.ndarray>

In [31]:
sr_0_no_chunks = fc_dataarray = None

In [32]:
df = combined.to_dataframe()

In [33]:
df

,features_id,blue_mean,green_mean,red_mean,nir_mean,ndvi_mean,ndvi_max,ndvi_min,ndvi_argmax
z,,,,,,,,,
0,523964.0,531.0,776.0,661.0,3091.0,6674.0,8174.0,2621.0,2767.0
1,523964.0,530.0,798.0,645.0,3221.0,6763.0,7984.0,4301.0,2767.0
2,523964.0,497.0,695.0,593.0,2854.0,6683.0,8255.0,3457.0,4884.0
3,523964.0,546.0,820.0,666.0,3175.0,6633.0,8060.0,3746.0,2767.0
4,523964.0,474.0,678.0,599.0,2874.0,6582.0,8485.0,3200.0,5229.0
5,523964.0,481.0,703.0,580.0,3015.0,6867.0,8243.0,3705.0,4884.0
6,523964.0,539.0,826.0,668.0,3302.0,6670.0,8253.0,4298.0,2767.0
7,523964.0,523.0,794.0,626.0,3116.0,6706.0,8491.0,4541.0,2767.0
8,425689.0,465.0,767.0,570.0,3136.0,7438.0,8710.0,-1358.0,5359.0


In [34]:
combined = None

In [35]:
gc.collect()

14

when running the following cell the memory use is the maximum of the whole process

In [36]:
df = df.sort_values(by='features_id')
df1 = df.groupby('features_id').agg({'ndvi_argmax': lambda x: scipy.stats.mode(x)[0]}).reset_index()
df2 = pd.merge(df1, df, on=['features_id', 'ndvi_argmax'], how='left')
df = None
gc.collect()
df3 = df2.groupby('features_id').agg({'blue_mean': 'mean', 'green_mean': 'mean', 'red_mean': 'mean', 'nir_mean': 'mean', 'ndvi_min': 'mean', 'ndvi_max': 'mean', 'ndvi_mean': 'mean'}).reset_index()
df2 = None
gc.collect()
dff = pd.merge(df1, df3, on=['features_id'], how='left')

In [37]:
dff

,features_id,ndvi_argmax,blue_mean,green_mean,red_mean,nir_mean,ndvi_min,ndvi_max,ndvi_mean
0,111.0,2724.0,341.640000,536.960000,368.760000,2763.240000,2718.320000,8962.000000,7336.640000
1,170.0,2724.0,582.666667,783.333333,744.555556,2434.666667,1930.333333,7773.555556,5100.555556
2,261.0,2767.0,626.859375,804.765625,724.312500,2642.843750,1894.750000,8240.062500,5706.984375
3,275.0,2767.0,689.411765,911.745098,987.921569,2443.568627,1696.980392,8215.901961,4176.960784
4,540.0,2767.0,723.969697,860.454545,849.969697,2100.030303,1653.303030,8503.848485,4185.303030
5,751.0,2767.0,595.533333,822.866667,596.733333,3208.733333,1747.733333,8902.533333,6877.800000
6,1158.0,5100.0,718.666667,887.333333,660.000000,3224.666667,1519.000000,8776.666667,7007.000000
7,1372.0,2767.0,562.400000,740.133333,544.800000,2638.066667,2379.300000,8750.366667,6618.500000
8,2698.0,2724.0,489.193548,660.612903,588.451613,2449.677419,2960.580645,8349.548387,5962.387097
9,3341.0,2508.0,834.309524,1028.785714,1127.738095,2478.357143,1790.023810,6984.666667,3794.809524


In [1]:
num_samples

In [38]:
df1 = df3 = None
gc.collect()

35

In [40]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], crs = crs)

In [41]:
gdf = gdf.rename(columns={'id':'features_id'})


In [42]:
gdf_df_join = gdf.merge(dff, on='features_id', how='right')


In [43]:
gdf_df_join

,geometry,features_id,ndvi_argmax,blue_mean,green_mean,red_mean,nir_mean,ndvi_min,ndvi_max,ndvi_mean
0,"POLYGON ((3039870.000 873300.000, 3039920.000 ...",536977,2767.0,727.788462,1025.076923,842.826923,3583.076923,2590.711538,7829.461538,6326.403846
1,"POLYGON ((3075720.000 873110.000, 3075770.000 ...",535938,2767.0,493.927273,700.072727,528.654545,2887.654545,3448.945455,8834.890909,6853.636364
2,"POLYGON ((3039430.000 872950.000, 3039440.000 ...",534765,4884.0,593.363636,780.242424,741.424242,2580.484848,2524.393939,8161.121212,5322.030303
3,"POLYGON ((3031620.000 872880.000, 3031640.000 ...",533861,5229.0,763.052632,983.105263,837.684211,3067.842105,-252.842105,8431.578947,6083.894737
4,"POLYGON ((3054970.000 872890.000, 3055020.000 ...",533389,2767.0,411.188679,567.905660,381.150943,3178.207547,3775.132075,9066.037736,7809.377358
5,"POLYGON ((3056450.000 872540.000, 3056470.000 ...",531284,2767.0,519.500000,685.333333,518.333333,3341.833333,1931.166667,9004.000000,7521.833333
6,"POLYGON ((3073030.000 872470.000, 3073040.000 ...",530710,2724.0,515.500000,713.000000,609.000000,2849.000000,2462.500000,8366.500000,6375.500000
7,"POLYGON ((3042650.000 872310.000, 3042670.000 ...",528935,4884.0,554.500000,758.833333,721.333333,2702.500000,2445.833333,7964.500000,5939.333333
8,"POLYGON ((3067270.000 872300.000, 3067280.000 ...",527633,2508.0,834.272727,980.204545,1025.272727,2229.227273,-559.250000,7466.750000,3868.386364
9,"POLYGON ((3044680.000 872200.000, 3044710.000 ...",527794,4884.0,861.500000,1045.750000,985.250000,2826.750000,1281.250000,8566.250000,5422.750000


In [44]:
fc = dff = gdf = None
gc.collect()

49

**NOTE:** First we save gdf_df_join of each dctile and once we have all of them in shapefiles, then we load them and make one out of them to fit the model

In [ ]:
if fit:
    gdf_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/gdf_df_join_frac_{}_{}_2019.shp'.format(str(frac), tile)
    gdf_filename
    gdf_df_join.to_file(gdf_filename)

## 2) Load gdf_df_join of all dctiles to fit the model
if you are making predictions skip this part and go to **4) Make predictions**

In [31]:
tilesfiles = ['/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/gdf_df_join_frac_{}_{}_2019.shp'.format(str(frac), tile)
              for tile in tiles ]

In [33]:
tables = []
for tile in tilesfiles:
    tables.append(gpd.read_file(tile))

In [34]:
tables[0].tail()

,features_i,ndvi_argma,blue_mean,green_mean,red_mean,nir_mean,ndvi_min,ndvi_max,ndvi_mean,geometry
35127,485,2490.0,471.526316,723.631579,479.000000,3115.078947,2558.184211,8767.631579,7436.842105,"POLYGON ((2964160.000 926220.000, 2964200.000 ..."
35128,519,3743.0,714.190476,952.619048,966.619048,2715.904762,2430.142857,7236.666667,4824.666667,"POLYGON ((2965740.000 926180.000, 2965770.000 ..."
35129,539,2490.0,689.125000,968.625000,810.500000,3527.125000,1309.875000,8585.875000,6335.000000,"POLYGON ((2966800.000 926130.000, 2966880.000 ..."
35130,650,4840.0,608.909091,849.000000,771.454545,2880.545455,1964.454545,8220.272727,5790.181818,"POLYGON ((2973450.000 926160.000, 2973490.000 ..."
35131,689,2767.0,553.200000,798.200000,696.400000,3394.200000,3953.600000,8624.000000,6552.200000,"POLYGON ((2975850.000 926170.000, 2975860.000 ..."


In [10]:
import pandas as pd

In [43]:
gdf_df_join = pd.concat(tables).set_index('features_i').reset_index().sort_index()
tables = []
gdf_df_join.tail()

,features_i,ndvi_argma,blue_mean,green_mean,red_mean,nir_mean,ndvi_min,ndvi_max,ndvi_mean,geometry
233582,472,2508.0,785.877778,929.544444,1021.100000,2139.233333,1331.944444,7280.477778,3583.133333,"POLYGON ((3063050.000 826200.000, 3063070.000 ..."
233583,491,2724.0,922.000000,1104.000000,1187.000000,2218.000000,1728.000000,4850.000000,3014.000000,"POLYGON ((3064130.000 826090.000, 3064150.000 ..."
233584,558,2767.0,614.000000,784.242857,667.342857,2325.742857,1159.442857,8560.700000,5695.857143,"POLYGON ((3068350.000 826220.000, 3068370.000 ..."
233585,594,2724.0,778.266667,931.100000,1049.300000,2099.766667,1576.166667,7159.166667,3281.300000,"POLYGON ((3070860.000 826120.000, 3070920.000 ..."
233586,618,2724.0,726.537736,857.924528,861.867925,1943.537736,1055.264151,7664.141509,3905.235849,"POLYGON ((3072800.000 826160.000, 3072840.000 ..."


In [44]:
gdf_df_join

,features_i,ndvi_argma,blue_mean,green_mean,red_mean,nir_mean,ndvi_min,ndvi_max,ndvi_mean,geometry
0,501660,4910.0,756.590909,1031.363636,833.000000,3298.000000,2112.227273,7851.772727,6098.727273,"POLYGON ((2937380.000 976140.000, 2937480.000 ..."
1,501731,3441.0,912.312500,1128.437500,1001.687500,3294.250000,1878.875000,7760.250000,5560.812500,"POLYGON ((2938190.000 976140.000, 2938290.000 ..."
2,501803,5488.0,732.230769,905.307692,881.615385,2306.384615,830.538462,6894.538462,4523.461538,"POLYGON ((2956840.000 976140.000, 2956920.000 ..."
3,501861,2983.0,802.555556,997.222222,860.444444,3106.000000,768.777778,8479.666667,6024.222222,"POLYGON ((2960830.000 976140.000, 2960930.000 ..."
4,501862,2983.0,880.666667,1088.133333,1007.333333,2916.333333,1017.066667,8724.000000,5120.200000,"POLYGON ((2961940.000 976140.000, 2962030.000 ..."
5,501759,2767.0,873.416667,1083.500000,940.500000,3088.583333,427.666667,8220.166667,5721.750000,"POLYGON ((2962850.000 976140.000, 2962890.000 ..."
6,501707,3112.0,1101.000000,1339.000000,1260.000000,3486.750000,361.250000,8316.500000,5127.750000,"POLYGON ((2963320.000 976140.000, 2963330.000 ..."
7,501812,2767.0,1151.347826,1386.173913,1475.739130,2591.478261,893.043478,6366.043478,3033.086957,"POLYGON ((2963970.000 976140.000, 2964160.000 ..."
8,501769,2983.0,916.200000,1109.300000,976.750000,2774.650000,607.950000,8676.150000,5361.150000,"POLYGON ((2965340.000 976140.000, 2965450.000 ..."
9,501825,2767.0,996.000000,1228.500000,1232.000000,2859.000000,1531.500000,7139.250000,4204.250000,"POLYGON ((2973960.000 976140.000, 2973980.000 ..."


## 3) Unsupervised clustering

In [50]:
kmeans = KMeans(n_clusters=n_clusters)

In [51]:
#kmeans.fit(gdf_df_join[['ndvi_argma','blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'ndvi_min', 'ndvi_max', 'ndvi_mean']].fillna(0).values)
kmeans.fit(gdf_df_join[['blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'ndvi_min', 'ndvi_max', 'ndvi_mean']].fillna(0).values)
#kmeans.fit(gdf_df_join[['ndvi_min', 'ndvi_max', 'ndvi_mean']].fillna(0).values)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=30, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [52]:
kmeans.cluster_centers_

array([[  666.99388161,   884.21252267,   743.46670012,  3049.75430904,
         2009.79558889,  8395.79711149,  6199.44343589],
       [  624.23115739,   498.17119207,   398.95202147,   334.63604837,
        -4309.3187608 ,   958.49879   , -1306.01920431],
       [ 1057.23390299,  1338.9089062 ,  1513.65557421,  2596.17003358,
          701.06493082,  5866.49711996,  2750.3928255 ],
       [  555.70075599,   765.4283192 ,   547.11762762,  3339.6264008 ,
         2286.79644852,  8855.92477015,  7326.10674996],
       [  712.07118204,   922.34693639,   920.91474656,  2522.01247749,
         2388.29404733,  6990.32594218,  4714.82178724],
       [  518.92028611,   696.52654652,   660.35494215,  2145.60613427,
         1154.46736711,  7792.74447508,  5478.2906538 ],
       [ 1157.86857602,  1431.54490824,  1624.54660288,  2445.47886841,
          720.79145535,  4007.56097091,  2086.76381106],
       [  488.06464823,   703.2973028 ,   482.10851781,  3335.36667944,
         3500.51567563,  

In [54]:
#kmeans_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_{}_frac_{}_2019_ndvis.pkl'.format(str(n_clusters), str(frac))
kmeans_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_07/kmeans_ncls_{}_frac_{}_2019_no_argmax.pkl'.format(str(n_clusters), str(frac))
#kmeans_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_{}_frac_{}_2019.pkl'.format(str(n_clusters), str(frac))
kmeans_filename

'/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_07/kmeans_ncls_30_frac_0.07_2019_no_argmax.pkl'

In [55]:
# save kmeans model
pickle.dump(kmeans, open(kmeans_filename, "wb"))

## 4) Make predictions

In [102]:
frac = 0.35  # 0.07
n_clusters = 30
kmeans_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_{}_frac_{}_2019_ndvis.pkl'.format(str(n_clusters), str(frac))
#kmeans_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_{}_frac_{}_2019_no_argmax.pkl'.format(str(n_clusters), str(frac))
#kmeans_filename = '/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_{}_frac_{}_2019.pkl'.format(str(n_clusters), str(frac))
kmeans_filename

'/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_30_frac_0.35_2019_ndvis.pkl'

In [103]:
kmeans = pickle.load(open(kmeans_filename, "rb"))


In [48]:
gdf_df_join = gdf_df_join.rename(columns={'ndvi_argmax': 'ndvi_argma', 'features_id': 'features_i'})
gdf_df_join.head()

,geometry,features_i,ndvi_argma,blue_mean,green_mean,red_mean,nir_mean,ndvi_min,ndvi_max,ndvi_mean
0,"POLYGON ((3039870.000 873300.000, 3039920.000 ...",536977,2767.0,727.788462,1025.076923,842.826923,3583.076923,2590.711538,7829.461538,6326.403846
1,"POLYGON ((3075720.000 873110.000, 3075770.000 ...",535938,2767.0,493.927273,700.072727,528.654545,2887.654545,3448.945455,8834.890909,6853.636364
2,"POLYGON ((3039430.000 872950.000, 3039440.000 ...",534765,4884.0,593.363636,780.242424,741.424242,2580.484848,2524.393939,8161.121212,5322.030303
3,"POLYGON ((3031620.000 872880.000, 3031640.000 ...",533861,5229.0,763.052632,983.105263,837.684211,3067.842105,-252.842105,8431.578947,6083.894737
4,"POLYGON ((3054970.000 872890.000, 3055020.000 ...",533389,2767.0,411.188679,567.905660,381.150943,3178.207547,3775.132075,9066.037736,7809.377358


In [104]:
preds = kmeans.predict(gdf_df_join[['ndvi_min', 'ndvi_max', 'ndvi_mean']].fillna(0).values)
#preds = kmeans.predict(gdf_df_join[['blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'ndvi_min', 'ndvi_max', 'ndvi_mean']].fillna(0).values)
#preds = kmeans.predict(gdf_df_join[['ndvi_argma','blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'ndvi_min', 'ndvi_max', 'ndvi_mean']].fillna(0).values)

In [105]:
gdf_df_join['preds'] = preds

In [106]:
gdf_df_cluster = gdf_df_join.drop(['ndvi_argma','blue_mean', 'green_mean', 'red_mean', 'nir_mean', 'ndvi_min', 'ndvi_max', 'ndvi_mean'],axis=1)

In [107]:
gdf_df_cluster.head()

,geometry,features_i,preds
0,"POLYGON ((3039870.000 873300.000, 3039920.000 ...",536977,22
1,"POLYGON ((3075720.000 873110.000, 3075770.000 ...",535938,29
2,"POLYGON ((3039430.000 872950.000, 3039440.000 ...",534765,3
3,"POLYGON ((3031620.000 872880.000, 3031640.000 ...",533861,8
4,"POLYGON ((3054970.000 872890.000, 3055020.000 ...",533389,29


In [108]:
gpds = gpd.read_file(seg_intersec_path)
gpds = gpds.rename(columns={'id':'features_i'})
gpds_cluster = gpds.merge(gdf_df_cluster, on=['features_i', 'geometry'], how='inner')

In [109]:
gpds_cluster

,features_i,geometry,preds
0,467778,"POLYGON ((3038300.000 866500.000, 3038320.000 ...",3
1,487113,"POLYGON ((3038380.000 868390.000, 3038420.000 ...",3
2,493320,"POLYGON ((3038250.000 869050.000, 3038270.000 ...",3
3,491085,"POLYGON ((3038310.000 868770.000, 3038330.000 ...",15
4,521829,"POLYGON ((3039090.000 871730.000, 3039100.000 ...",22
5,474805,"POLYGON ((3038800.000 867510.000, 3038810.000 ...",22
6,490672,"POLYGON ((3038340.000 868870.000, 3038390.000 ...",3
7,536977,"POLYGON ((3039870.000 873300.000, 3039920.000 ...",22
8,407192,"POLYGON ((3028700.000 860760.000, 3028730.000 ...",22
9,511008,"POLYGON ((3038900.000 870640.000, 3038970.000 ...",15


In [110]:
name_file_gpds_cluster = kmeans_filename.split('.pkl')[0] + '_' + tile + '_cluster_segments.shp'
name_file_gpds_cluster

'/shared_volume/tasks/2019/clusterization_Veracruz/gdf_df_joins_frac_35/kmeans_ncls_30_frac_0.35_2019_ndvis_41_-32_cluster_segments.shp'

In [111]:
gpds_cluster.to_file(name_file_gpds_cluster)

In [112]:
gpds_cluster = gdf_df_cluster = None

## 5) Run shell script to add type of crops to *cluster_segments_.shp
The shell script is in antares3-sandbox/scripts/get_intersected_segments.py 

**Usage**  
./get_intersected_segments.py polygons_filenames_list.txt crops_points_file.shp results_directory

## 6) make csv's after shell script

In [115]:
from os import listdir

In [131]:
path = '/shared_volume/tasks/2019/clusterization_Veracruz/results_frac_35'

In [132]:
shapefiles = [path + '/' + f for f in listdir(path) if f.split('.')[-1] == 'shp' ]


In [133]:
for shpf in shapefiles:
    shapef = gpd.read_file(shpf)
    shapef = shapef.drop(['geometry'],axis=1)
    shapef.to_csv(shpf.split('.shp')[0] + '.csv' , index=False)